# Imports and Setup
This section handles the initial setup. It imports the required libraries and establishes a read-only connection to the DuckDB database that contains the clean, structured data from our ETL process.

In [1]:
import duckdb
import pandas as pd
from pathlib import Path

# --- Configuration ---
# Point to the database file you created in the ETL step
OUTPUT_DIR = Path("../output")
DB_FILE = OUTPUT_DIR / "synthea_fhir.duckdb"

# --- Connect to the database ---
# We're connecting in read-only mode since we are just querying, not writing new tables.
print(f"Connecting to DuckDB database: {DB_FILE}")
con = duckdb.connect(database=str(DB_FILE), read_only=True)

# Set pandas to display more columns for our sanity checks
pd.set_option('display.max_columns', 50)

Connecting to DuckDB database: ../output/synthea_fhir.duckdb


# Identify Index Admissions
The first step in our analysis is to create a cohort of relevant hospitalizations. This query filters the millions of encounters down to only inpatient admissions, which will serve as the starting point (or "index admissions") for our prediction task.

In [2]:
# SQL query to select all inpatient encounters
sql_query = """
SELECT
    Id AS encounter_id,
    Patient AS patient_id,
    Start AS admission_date,
    Stop AS discharge_date
FROM
    encounters
WHERE
    EncounterClass = 'IMP'
"""

# Execute and fetch into a pandas DataFrame
index_admissions_df = con.execute(sql_query).fetchdf()

🩺 Sanity Check
Let's verify that we've correctly filtered the data. We should see a reasonable number of inpatient stays and the dates should look correct.

In [3]:
print(f"Found {len(index_admissions_df):,} total inpatient admissions.")
print("First 5 index admissions:")
display(index_admissions_df.head())

# Check for any admissions with null discharge dates, as these are problematic
print(f"\nAdmissions with missing discharge dates: {index_admissions_df['discharge_date'].isnull().sum()}")

Found 104,068 total inpatient admissions.
First 5 index admissions:


,encounter_id,patient_id,admission_date,discharge_date
0,d9f75434-be7a-663f-8702-8ac72afe10fd,7b0e0003-89c8-6020-da41-033cf174c76f,1997-05-18 10:35:30,1997-05-23 08:41:12
1,58fa79a0-fb0a-9eb3-562a-ece296e084f6,e8e952cb-c390-f661-ad04-b95818ec40a4,1994-05-21 00:52:34,1994-05-25 19:53:04
2,8be2ca69-121a-a779-3ac9-2ecb97ad8908,49b95517-d094-9b08-3982-c86d39450368,2025-04-03 20:25:55,2025-04-04 20:40:55
3,ee89a84f-9b21-fd36-2419-9d85ce9f8b97,4c161043-b934-badf-3375-019f05f72cb1,2016-03-08 00:08:09,2016-03-22 01:08:09
4,54ce90f5-1bcd-0379-2c0c-d000a57e390c,4c161043-b934-badf-3375-019f05f72cb1,2016-11-08 00:08:09,2016-11-13 00:08:09



Admissions with missing discharge dates: 0


# Engineer the Target Variable (readmitted_within_30_days)
This is the most critical step, where we create our prediction target. Using a SQL window function (LEAD), this query looks forward in time for each patient to find their next hospitalization. It then calculates the time between discharge and that next admission to create the final readmitted_within_30_days label.

In [4]:
# %%
# This query calculates the time to the next admission and creates the readmission flag.
sql_query = """
WITH PatientAdmissions AS (
    -- 1. Get all inpatient admissions and order them by patient and date
    SELECT
        Id AS encounter_id,
        Patient AS patient_id,
        Start AS admission_date,
        Stop AS discharge_date,
        -- 2. Use the LEAD window function to get the start date of the *next* admission
        -- for the same patient. If there is no next admission, it will be NULL.
        LEAD(Start, 1) OVER(PARTITION BY Patient ORDER BY Start) AS next_admission_date
    FROM
        encounters
    WHERE
        EncounterClass = 'IMP'
)
-- 3. Now, calculate the difference and create the flag
SELECT
    encounter_id,
    patient_id,
    admission_date,
    discharge_date,
    next_admission_date,
    -- Calculate days between discharge and the next admission
    DATE_DIFF('day', discharge_date, next_admission_date) AS days_to_next_admission,
    -- Create the binary target variable
    CASE
        WHEN DATE_DIFF('day', discharge_date, next_admission_date) <= 30 THEN 1
        ELSE 0
    END AS readmitted_within_30_days
FROM
    PatientAdmissions
-- We can't predict readmission for ongoing hospitalizations
WHERE
    discharge_date IS NOT NULL
"""

# This will be our main dataset that we'll add features to
readmissions_df = con.execute(sql_query).fetchdf()

🩺 Sanity Check
Let's check the distribution of our new target variable. We should also manually inspect a few positive cases to be sure our logic is correct.

In [5]:
# %%
# Check the distribution of the target variable
print("Readmission distribution:")
print(readmissions_df['readmitted_within_30_days'].value_counts(normalize=True))

# Find a patient who was readmitted and verify the dates manually
readmitted_patient_example = readmissions_df[readmissions_df['readmitted_within_30_days'] == 1].head(5)
print("\nExample of patients who were readmitted:")
display(readmitted_patient_example)

Readmission distribution:
readmitted_within_30_days
0    0.821905
1    0.178095
Name: proportion, dtype: float64

Example of patients who were readmitted:


,encounter_id,patient_id,admission_date,discharge_date,next_admission_date,days_to_next_admission,readmitted_within_30_days
6,763b47c1-3bab-7793-c832-e7b195de756d,ecee60d1-63e3-3f47-81dd-9138e9f9a64d,2019-08-24 06:51:42,2019-08-25 06:51:42,2019-08-24 13:03:15,-1,1
7,69cc3e3d-b497-f1f9-669b-15a784d98838,ecee60d1-63e3-3f47-81dd-9138e9f9a64d,2019-08-24 13:03:15,2019-08-26 15:10:18,2019-08-26 14:55:18,0,1
35,aedd0167-e8ef-548c-d4dd-04d6aa2cb645,eda9dd60-455d-d26b-b5cd-733738f9acdc,2018-07-27 03:22:07,2018-08-06 04:36:07,2018-09-05 11:36:07,30,1
36,2bd0dbbe-6302-15ae-7362-8924e228804f,eda9dd60-455d-d26b-b5cd-733738f9acdc,2018-09-05 11:36:07,2018-09-11 03:29:07,2018-10-10 23:29:07,29,1
37,538df773-2de5-9a96-8543-1d36223166b0,eda9dd60-455d-d26b-b5cd-733738f9acdc,2018-10-10 23:29:07,2018-10-24 06:03:07,2018-11-22 23:03:07,29,1


# Add Demographics and Admission-Level Features
With our target variable defined, we begin building the feature set. This query joins the index admissions with patient and encounter data to add demographic information (like age, race, gender) and key details about the hospitalization itself (like length of stay, payer, and primary diagnosis).

In [6]:
sql_query = """
SELECT
    -- Key identifiers from our readmissions table
    readmissions.encounter_id,
    readmissions.patient_id,

    -- The target variable
    readmissions.readmitted_within_30_days,

    -- Feature 1: Length of Stay (in days)
    DATE_DIFF('day', readmissions.admission_date, readmissions.discharge_date) AS length_of_stay,

    -- Feature 2: Age at time of admission
    DATE_DIFF('year', patients.BirthDate, readmissions.admission_date) AS age_at_admission,

    -- Demographic features from the patients table
    patients.Gender AS gender,
    patients.Race AS race,
    patients.Marital AS marital_status,
    
    -- Admission details from the original encounters table
    enc.Description AS admission_reason,
    enc.ReasonDescription AS admission_reason_detail,
    enc.Payer AS payer,
    enc.Total_Claim_Cost AS total_claim_cost,
    patients.Income AS income,
    DAYNAME(readmissions.admission_date) AS admission_day_of_week,

    -- === NEW HIGH-CARDINALITY FEATURES ===

    -- 1. Primary Diagnosis Code (from Encounter ReasonCode)
    enc.ReasonCode AS primary_diagnosis_code,

    -- 2. Attending Provider ID
    enc.Provider AS provider_id

FROM
    -- Use our previously created DataFrame as a source table
    readmissions_df AS readmissions
LEFT JOIN
    patients ON readmissions.patient_id = patients.Id
LEFT JOIN
    encounters AS enc ON readmissions.encounter_id = enc.Id
"""

# Create our analytical base table
model_df = con.execute(sql_query).fetchdf()

In [7]:
# Create the new interaction feature by combining two columns
# We convert to string and fill NAs to prevent errors
model_df['payer_dx_interaction'] = (
    model_df['payer'].astype(str).fillna('unknown') + '_' + 
    model_df['primary_diagnosis_code'].astype(str).fillna('unknown')
)

print("New high-cardinality features added:")
display(model_df[['primary_diagnosis_code', 'provider_id', 'payer_dx_interaction']].head())

New high-cardinality features added:


,primary_diagnosis_code,provider_id,payer_dx_interaction
0,399261000,us-npi|9999868992,Blue Cross Blue Shield_399261000
1,399261000,us-npi|9999868992,Blue Cross Blue Shield_399261000
2,183996000,us-npi|9999952390,NO_INSURANCE_183996000
3,840539006,us-npi|9999943399,Anthem_840539006
4,88805009,us-npi|9999868992,Medicare_88805009


🩺 Sanity Check
Let's inspect the resulting dataframe. The row count should be the same as our readmissions_df. We should also check the distributions of our new numeric features like age_at_admission and length_of_stay to see if they are reasonable.

In [8]:
print(f"Total rows in our model dataset: {len(model_df):,}")
print("First 5 rows of the feature table:")
display(model_df.head())

print("\nStatistical summary of numeric features:")
display(model_df[['length_of_stay', 'age_at_admission']].describe())

Total rows in our model dataset: 104,068
First 5 rows of the feature table:


,encounter_id,patient_id,readmitted_within_30_days,length_of_stay,age_at_admission,gender,race,marital_status,admission_reason,admission_reason_detail,payer,total_claim_cost,income,admission_day_of_week,primary_diagnosis_code,provider_id,payer_dx_interaction
0,ef5d7e9f-956d-2b7a-a4a6-c632f3b40cf9,1f167660-c201-84b9-f69a-16d19d622bfa,1,7,60,male,White,M,Patient transfer to intensive care unit (proce...,History of coronary artery bypass grafting (si...,Blue Cross Blue Shield,67106.281250,99505,Wednesday,399261000,us-npi|9999868992,Blue Cross Blue Shield_399261000
1,3c5e1be2-468a-e4d8-11f2-e767d59482d5,1f167660-c201-84b9-f69a-16d19d622bfa,0,1,60,male,White,M,Admission to ward (procedure),History of coronary artery bypass grafting (si...,Blue Cross Blue Shield,2176.229980,99505,Wednesday,399261000,us-npi|9999868992,Blue Cross Blue Shield_399261000
2,6f06a6aa-a1da-bcd6-a43f-ddbbd638947c,68bfa4c8-e3ac-6dda-a02a-a225c0b2c9a5,0,1,35,female,White,M,Admission to surgical department (procedure),Sterilization requested (situation),NO_INSURANCE,4495.109863,119210,Saturday,183996000,us-npi|9999952390,NO_INSURANCE_183996000
3,e2477992-082b-69ca-3152-6fecf4442626,45ccdf82-db5c-3947-2d97-ee18a8a9c4e3,0,12,53,female,White,W,Hospital admission for isolation (procedure),Disease caused by severe acute respiratory syn...,Anthem,37806.300781,60503,Saturday,840539006,us-npi|9999943399,Anthem_840539006
4,735f3287-d205-1ec8-9668-fcdac03f306a,a5a065d0-a35b-d8e3-fda6-0e92c0a40926,0,1,63,female,White,M,Admission to intensive care unit (procedure),Chronic congestive heart failure (disorder),Medicare,4361.350098,49355,Monday,88805009,us-npi|9999868992,Medicare_88805009



Statistical summary of numeric features:


,length_of_stay,age_at_admission
count,104068.000000,104068.000000
mean,6.917170,50.162759
std,14.102942,19.497225
min,0.000000,0.000000
25%,1.000000,36.000000
50%,4.000000,52.000000
75%,8.000000,65.000000
max,3306.000000,111.000000


# Engineer Historical Features
To capture a patient's recent history, this advanced SQL query calculates a key historical feature. For each hospitalization, it looks back over the previous year and counts the number of prior inpatient stays for that same patient, giving us a measure of their recent healthcare utilization.

In [9]:
# This query is more complex. It performs a "self-join" on the encounters
# table to count previous visits for each index admission.
sql_query = """
SELECT
    index_admission.Id AS encounter_id,
    -- Count all previous inpatient admissions within the last 365 days
    COUNT(prior_admissions.Id) AS prior_admissions_last_year
FROM
    encounters AS index_admission
LEFT JOIN
    encounters AS prior_admissions
ON
    -- Must be the same patient
    index_admission.Patient = prior_admissions.Patient
    -- The prior admission must have happened BEFORE the index one
    AND prior_admissions.Start < index_admission.Start
    -- And it must be within the last year
    AND DATE_DIFF('day', prior_admissions.Start, index_admission.Start) <= 365
    -- And it must also be an inpatient admission
    AND prior_admissions.EncounterClass = 'IMP'
WHERE
    -- We only need to calculate this for our index admissions
    index_admission.EncounterClass = 'IMP'
GROUP BY
    index_admission.Id;
"""

prior_admissions_df = con.execute(sql_query).fetchdf()

# Now, merge this back into our main model dataframe
model_df = pd.merge(model_df, prior_admissions_df, on='encounter_id', how='left')

# Fill any NaNs that might result from the merge with 0
model_df['prior_admissions_last_year'] = model_df['prior_admissions_last_year'].fillna(0)

🩺 Sanity Check
Let's check the new column. The values should be integers, and the distribution should make sense (most patients will likely have 0 or 1 prior admissions).

In [10]:
print("Distribution of prior admissions in the last year:")
print(model_df['prior_admissions_last_year'].value_counts().sort_index())

display(model_df.head())

Distribution of prior admissions in the last year:
prior_admissions_last_year
0     66546
1     10832
2      3184
3      1464
4       860
5       804
6       928
7      1214
8      3005
9     10674
10     1931
11     1944
12      602
13       64
14       16
Name: count, dtype: int64


,encounter_id,patient_id,readmitted_within_30_days,length_of_stay,age_at_admission,gender,race,marital_status,admission_reason,admission_reason_detail,payer,total_claim_cost,income,admission_day_of_week,primary_diagnosis_code,provider_id,payer_dx_interaction,prior_admissions_last_year
0,ef5d7e9f-956d-2b7a-a4a6-c632f3b40cf9,1f167660-c201-84b9-f69a-16d19d622bfa,1,7,60,male,White,M,Patient transfer to intensive care unit (proce...,History of coronary artery bypass grafting (si...,Blue Cross Blue Shield,67106.281250,99505,Wednesday,399261000,us-npi|9999868992,Blue Cross Blue Shield_399261000,0
1,3c5e1be2-468a-e4d8-11f2-e767d59482d5,1f167660-c201-84b9-f69a-16d19d622bfa,0,1,60,male,White,M,Admission to ward (procedure),History of coronary artery bypass grafting (si...,Blue Cross Blue Shield,2176.229980,99505,Wednesday,399261000,us-npi|9999868992,Blue Cross Blue Shield_399261000,1
2,6f06a6aa-a1da-bcd6-a43f-ddbbd638947c,68bfa4c8-e3ac-6dda-a02a-a225c0b2c9a5,0,1,35,female,White,M,Admission to surgical department (procedure),Sterilization requested (situation),NO_INSURANCE,4495.109863,119210,Saturday,183996000,us-npi|9999952390,NO_INSURANCE_183996000,0
3,e2477992-082b-69ca-3152-6fecf4442626,45ccdf82-db5c-3947-2d97-ee18a8a9c4e3,0,12,53,female,White,W,Hospital admission for isolation (procedure),Disease caused by severe acute respiratory syn...,Anthem,37806.300781,60503,Saturday,840539006,us-npi|9999943399,Anthem_840539006,0
4,735f3287-d205-1ec8-9668-fcdac03f306a,a5a065d0-a35b-d8e3-fda6-0e92c0a40926,0,1,63,female,White,M,Admission to intensive care unit (procedure),Chronic congestive heart failure (disorder),Medicare,4361.350098,49355,Monday,88805009,us-npi|9999868992,Medicare_88805009,0


# Engineer Clinical Features (Diagnoses, Procedures, Medications)
This section adds features that quantify the clinical complexity of each hospital stay. By grouping the conditions, procedures, and medications tables, we calculate the total number of diagnoses recorded, procedures performed, and medications administered during each specific admission.

In [11]:
# ----- 1. Number of Diagnoses (Conditions) -----
print("Engineering feature: Number of Diagnoses...")
sql_diagnoses = """
SELECT
    Encounter AS encounter_id,
    COUNT(Code) AS num_diagnoses
FROM
    conditions
GROUP BY
    Encounter;
"""
diagnoses_df = con.execute(sql_diagnoses).fetchdf()

# Merge into the main dataframe
model_df = pd.merge(model_df, diagnoses_df, on='encounter_id', how='left')
model_df['num_diagnoses'] = model_df['num_diagnoses'].fillna(0)


# ----- 2. Number of Procedures -----
print("Engineering feature: Number of Procedures...")
sql_procedures = """
SELECT
    Encounter AS encounter_id,
    COUNT(Code) AS num_procedures
FROM
    procedures
GROUP BY
    Encounter;
"""
procedures_df = con.execute(sql_procedures).fetchdf()

# Merge into the main dataframe
model_df = pd.merge(model_df, procedures_df, on='encounter_id', how='left')
model_df['num_procedures'] = model_df['num_procedures'].fillna(0)


# ----- 3. Number of Medications -----
print("Engineering feature: Number of Medications...")
sql_medications = """
SELECT
    Encounter AS encounter_id,
    COUNT(Code) AS num_medications
FROM
    medications
GROUP BY
    Encounter;
"""
medications_df = con.execute(sql_medications).fetchdf()

# Merge into the main dataframe
model_df = pd.merge(model_df, medications_df, on='encounter_id', how='left')
model_df['num_medications'] = model_df['num_medications'].fillna(0)

print("✅ Clinical features added successfully.")

Engineering feature: Number of Diagnoses...
Engineering feature: Number of Procedures...
Engineering feature: Number of Medications...
✅ Clinical features added successfully.


🩺 Sanity Check
Let's inspect our dataframe again to ensure the new columns have been added correctly. They should contain integer counts.

In [12]:
print("First 5 rows with new clinical features:")
display(model_df.head())

print("\nUpdated statistical summary:")
display(model_df[['length_of_stay', 'age_at_admission', 'prior_admissions_last_year', 'num_diagnoses', 'num_procedures', 'num_medications']].describe())

First 5 rows with new clinical features:


,encounter_id,patient_id,readmitted_within_30_days,length_of_stay,age_at_admission,gender,race,marital_status,admission_reason,admission_reason_detail,payer,total_claim_cost,income,admission_day_of_week,primary_diagnosis_code,provider_id,payer_dx_interaction,prior_admissions_last_year,num_diagnoses,num_procedures,num_medications
0,ef5d7e9f-956d-2b7a-a4a6-c632f3b40cf9,1f167660-c201-84b9-f69a-16d19d622bfa,1,7,60,male,White,M,Patient transfer to intensive care unit (proce...,History of coronary artery bypass grafting (si...,Blue Cross Blue Shield,67106.281250,99505,Wednesday,399261000,us-npi|9999868992,Blue Cross Blue Shield_399261000,0,0.0,24.0,0.0
1,3c5e1be2-468a-e4d8-11f2-e767d59482d5,1f167660-c201-84b9-f69a-16d19d622bfa,0,1,60,male,White,M,Admission to ward (procedure),History of coronary artery bypass grafting (si...,Blue Cross Blue Shield,2176.229980,99505,Wednesday,399261000,us-npi|9999868992,Blue Cross Blue Shield_399261000,1,0.0,4.0,0.0
2,6f06a6aa-a1da-bcd6-a43f-ddbbd638947c,68bfa4c8-e3ac-6dda-a02a-a225c0b2c9a5,0,1,35,female,White,M,Admission to surgical department (procedure),Sterilization requested (situation),NO_INSURANCE,4495.109863,119210,Saturday,183996000,us-npi|9999952390,NO_INSURANCE_183996000,0,1.0,2.0,0.0
3,e2477992-082b-69ca-3152-6fecf4442626,45ccdf82-db5c-3947-2d97-ee18a8a9c4e3,0,12,53,female,White,W,Hospital admission for isolation (procedure),Disease caused by severe acute respiratory syn...,Anthem,37806.300781,60503,Saturday,840539006,us-npi|9999943399,Anthem_840539006,0,4.0,27.0,0.0
4,735f3287-d205-1ec8-9668-fcdac03f306a,a5a065d0-a35b-d8e3-fda6-0e92c0a40926,0,1,63,female,White,M,Admission to intensive care unit (procedure),Chronic congestive heart failure (disorder),Medicare,4361.350098,49355,Monday,88805009,us-npi|9999868992,Medicare_88805009,0,0.0,5.0,0.0



Updated statistical summary:


,length_of_stay,age_at_admission,prior_admissions_last_year,num_diagnoses,num_procedures,num_medications
count,104068.000000,104068.000000,104068.000000,104068.000000,104068.000000,104068.000000
mean,6.917170,50.162759,2.039032,0.377984,9.333936,0.183995
std,14.102942,19.497225,3.557021,0.719728,15.938124,0.818711
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,36.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,52.000000,0.000000,0.000000,4.000000,0.000000
75%,8.000000,65.000000,2.000000,1.000000,12.000000,0.000000
max,3306.000000,111.000000,14.000000,8.000000,176.000000,9.000000


# Save the Final Dataset & Cleanup
The feature engineering process is now complete. This final step saves the resulting dataset—a single, clean table containing the target variable and all engineered features—to a Parquet file. This efficient format is ideal for the next stage of model training.

In [13]:
# Define the path for our final dataset
MODEL_DATA_FILE = OUTPUT_DIR / "readmissions_dataset.parquet"

# Save to Parquet
model_df.to_parquet(MODEL_DATA_FILE, index=False)

# Close the database connection
con.close()

print(f"✅ Successfully created and saved the feature table with {len(model_df):,} rows and {len(model_df.columns)} columns.")
print(f"Dataset saved to: {MODEL_DATA_FILE}")

✅ Successfully created and saved the feature table with 104,068 rows and 21 columns.
Dataset saved to: ../output/readmissions_dataset.parquet
